In [1]:
# This is a Python script designed to run in a Google Colab notebook.
# To use, copy this code into a Colab cell and run it.

# Install required libraries (uncomment if running in Colab)
# !pip install matplotlib numpy

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

# Simulation parameters
GRID_SIZE = 32  # 32x32 grid
D = GRID_SIZE * GRID_SIZE  # Ψ dimension (1024)
D_PHI = 2  # Φ dimension (x, y)
K = 4  # Z dimension (x, y, noise1, noise2)
FRAMES = 50  # Number of animation frames
SIGMA = 2.0  # Gaussian blob standard deviation
LEARNING_RATE = 0.02  # For Φ updates (small to ensure contractive updates)
MOVE_INTERVAL = 10  # Target moves every 10 frames

# Initialize environment and target
def setup():
    global target_pos, psi, phi
    # Target starts at a random position
    target_pos = [np.random.randint(0, GRID_SIZE), np.random.randint(0, GRID_SIZE)]
    # Initial sensory input Ψ: Gaussian blob centered at target_pos
    psi = generate_psi(target_pos)
    # Initial Φ: perturbed position
    phi = np.array([target_pos[0] + np.random.uniform(-5, 5), target_pos[1] + np.random.uniform(-5, 5)], dtype=float)
    return psi, phi, target_pos

def generate_psi(pos):
    """Generate sensory input Ψ: Gaussian blob centered at pos."""
    x, y = np.meshgrid(np.arange(GRID_SIZE), np.arange(GRID_SIZE), indexing='ij')
    psi = np.exp(-((x - pos[0])**2 + (y - pos[1])**2) / (2 * SIGMA**2))
    return psi / np.max(psi)  # Normalize to [0, 1]

def sample_z(phi):
    """Sample Z ~ p(Z|Φ): Gaussian centered on Φ."""
    mean = np.concatenate([phi, [0, 0]])  # Z = [x, y, noise1, noise2]
    cov = np.eye(K) * 0.1  # Small variance for contractive updates
    return np.random.multivariate_normal(mean, cov)

def decode_psi(z):
    """Generate predicted Ψ̂ ~ p(Ψ|Z): Gaussian blob centered at Z[0:2]."""
    return generate_psi(z[0:2])

def update_phi(psi_hat, psi, phi):
    """Update Φ using amortized inference operator F(Φ, Ψ̂): minimize error between Ψ̂ and Ψ."""
    # Reconstruction loss: mean squared error
    recon_error = psi - psi_hat
    mse = np.mean(recon_error**2)
    # Approximate gradient of MSE w.r.t. Φ[0:2] (x, y)
    x, y = np.meshgrid(np.arange(GRID_SIZE), np.arange(GRID_SIZE), indexing='ij')
    grad = np.zeros(D_PHI)
    grad[0] = np.sum(recon_error * (x - phi[0]) / (SIGMA**2)) / D  # Gradient w.r.t. x
    grad[1] = np.sum(recon_error * (y - phi[1]) / (SIGMA**2)) / D  # Gradient w.r.t. y
    new_phi = phi - LEARNING_RATE * grad
    # Clip to grid bounds
    new_phi = np.clip(new_phi, 0, GRID_SIZE-1)
    return new_phi, mse

# Animation update function
def update(frame):
    global target_pos, psi, phi
    # Move target every MOVE_INTERVAL frames
    if frame % MOVE_INTERVAL == 0:
        dx, dy = np.random.choice([-1, 0, 1]), np.random.choice([-1, 0, 1])
        new_pos = [target_pos[0] + dx, target_pos[1] + dy]
        new_pos = np.clip(new_pos, 0, GRID_SIZE-1)
        target_pos = new_pos.astype(int)
        psi = generate_psi(target_pos)

    # Sample Z and generate predicted Ψ̂
    z = sample_z(phi)
    psi_hat = decode_psi(z)
    # Update Φ using amortized inference
    phi, mse = update_phi(psi_hat, psi, phi)

    # Update plots
    ax[0].clear()
    ax[1].clear()
    ax[2].clear()
    ax[0].imshow(psi, cmap='viridis', origin='upper')
    ax[0].plot(target_pos[1], target_pos[0], 'r^', markersize=10)
    ax[0].set_title("True Context (Ψ)")
    ax[1].imshow(psi_hat, cmap='viridis', origin='upper')
    ax[1].plot(z[1], z[0], 'b*', markersize=10)
    ax[1].set_title("Predicted Context (Ψ̂)")
    ax[2].scatter([phi[1]], [phi[0]], c='green', marker='o', s=100)
    ax[2].set_title(f"Latent Φ (x={phi[0]:.1f}, y={phi[1]:.1f}, MSE={mse:.4f})")
    ax[2].set_xlim(0, GRID_SIZE)
    ax[2].set_ylim(0, GRID_SIZE)
    ax[2].set_xticks([])
    ax[2].set_yticks([])
    for a in ax:
        a.grid(False)
    return ax

# Set up the simulation
psi, phi, target_pos = setup()

# Create the animation
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ani = FuncAnimation(fig, update, frames=FRAMES, interval=200, repeat=False)
plt.close()  # Prevent duplicate display in Colab
from IPython.display import HTML
HTML(ani.to_jshtml())  # Display animation in Colab